<div style="display: flex; align-items: center;">
  <p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; text-align: left; margin-right: auto; line-height: 1.2;">
    <font color='black'><strong>Procesamiento Digital de Señales</strong><br>David Ochoa Cruz<br>R6575-2023</font>
  </p>
  <img src="logo-utn-frba-electronica.svg" alt="Logo UTN FRBA" width="200" height="40" style="flex-shrink: 0; margin-left: 10px;">
</div>

<p style="border: ridge white 2px; text-align: center;font-weight: italic; font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size:30px; background-color:#2CD546"><font color='white'>
    <strong>TS2: Algoritmo para calcular la DFT</strong>
</p>

<!DOCTYPE html>
<html>
<head>
<style>
  /* Estilos CSS aquí */
  body {
    font-family: Arial, sans-serif;
    background-color: white;
    color: black;
    margin: 20px;
    padding: 20px;
  }
</style>
</head>
<body>
<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">Desarrollar un algoritmo que calcule la transformada discreta de Fourier (DFT).</p>

$$X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j2\pi \cdot \frac{k \cdot n}{N}}$$
<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">Donde:</p>
<ul>
  <li style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">XX = mi_funcion_DFT( xx )</li>
  <li style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">xx: señal a analizar, una matriz (Nx1) de números reales.</li>
  <li style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">XX: DFT de xx, una matriz (Nx1) de números complejos..</li>
</ul>
<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;"><strong>Ayudas:</strong></p>
<br>
<div style="text-align: center;">
  <iframe width="800" height="450" src="https://www.youtube.com/embed/CJAZCYvigVc" frameborder="0" allowfullscreen></iframe>
</div>
<br>
<br>
<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">
Aprovechando que tenemos un generador de senoidales funcionando, y recordando que la DFT de una senoidal de frecuencia f0, será una delta de Kronecker posicionada en la frecuencia f0. Validen sus funciones con esta experiencia </p>

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;""><strong>Bonus:</strong></p>
<ul>
<li style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">💎 Probar las mismas experiencias con la <a href="https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html" target="_blank">transformada rápida de Fourier</a> (DFT rápida se llama FFT).</li>
<li style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace;">🤯 Calcular la DFT de una señal aleatoria, por ejemplo ruido uniforme de varianza σ² = 4. Pueden encontrar cómo depende la varianza de los parámetros de la distribución <a href="https://es.wikipedia.org/wiki/Distribuci%C3%B3n_uniforme_continua" target="_blank">aquí</a>.</li>
</ul>

<span style="border: 2px solid #FFFFFF; padding: 5px; display: inline-block; background-color: #2CD546; font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px; color: #FFFFFF; font-weight: bold;">
    Resolución
</span>
<br><br>
<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px; font-weight: bold;">
    Importamos librerías y módulos necesarios:
</p>


In [48]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image
from scipy import signal
import psdmodulos as psd

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Definición de parámetros de simulación:
</p>

In [49]:
fs = 1000                #   Frecuencia de muestreo en Hz
N  = fs                 #   Cantidad de muestras digitalizadas por el ADC (# muestras)
Ts = 1/fs               #   Periodo de muestreo

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Definimos los parámetros de la señal Senoidal:
</p>

In [50]:
frecuencia  = 50           # frecuencia de la señal en Hz
Amplitud    = np.sqrt(2)    # Amplitud máxiva en Volt
Phase       = np.pi/2.0     # Fase en radianes
ValorMedio  = 0             # Componente de continua
snr         = 40            # Relación señal-ruido en dB

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Creamos las funciones para generar la Señal y el Ruido
</p>

In [51]:
def GeneradorSenoidal(ff, AA, ph, dc, snr):
    tt, signal = generar_senoidal(ff, AA, ph, dc)
    ruido = generar_ruido_uniforme(signal, snr)
    return tt, signal + ruido

def generar_senoidal(ff, AA, ph, dc):
    tt = np.arange(start=0, stop=N*Ts, step=Ts)
    signal = AA * np.sin(2 * np.pi * ff * tt + ph) + dc
    return tt, signal

def generar_ruido_uniforme(signal, snr):
    pot_señal = np.var(signal)
    pot_ruido = pot_señal / (10**(snr / 10))  # Convertir SNR a escala lineal
    ruido = np.random.uniform(-np.sqrt(3*pot_ruido) , np.sqrt(3*pot_ruido) , len(signal))
    return ruido

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Función para graficar las Señales:
</p>

In [52]:
def plot_signal(tt, signal, title, color, xlabel='Tiempo [s]', ylabel='Amplitud [V]'):
    plt.figure(figsize=(12, 4))
    plt.plot(tt, signal,label=title, color=color)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, linestyle='dotted', color='gray')
    plt.savefig(f'{title}.png', bbox_inches='tight', transparent=True)
    plt.close()

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Configuración del estilo global de los gráficos:
</p>

In [53]:
plt.rcParams['savefig.transparent'] = True
plt.rcParams['axes.edgecolor'] = 'gray'
plt.rcParams['axes.labelcolor'] = 'gray'
plt.rcParams['xtick.color'] = 'gray'
plt.rcParams['ytick.color'] = 'gray'

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Generamos la Señal Senoidal con Ruido aleatorio uniforme
</p>

In [54]:
tt, xx = psd.SignalGenerator(Amplitud, frecuencia, Phase, ValorMedio, N, fs, signal='Senoidal', noise='normal', snr=snr)
psd.plot_signal(tt, xx, 'SenoidalRuido', color='blue', snr=snr)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.



	SNR = 39.95660767317072[dB]

	Potencia de la Señal:	 1.000000000000001[W]

	Potencia del Ruido:	 0.00010000000000000011[W]


<div style="text-align: center;">
  <span style="border: 2px solid white; padding: 5px; display: inline-block; background-color: blue; font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px; color: #FFFFFF; font-weight: bold;">
    Señal Senoidal + Ruido
  </span>
</div>
<div style="max-width: 100%; text-align: center;">
  <img src="SenoidalRuido.png" alt="Texto alternativo" style="max-width: 100%;">
</div>

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Definmos la el algoritmo de la DFT
</p>

In [55]:
def mi_funcion_DFT(xx):
    N = len(xx)  # Número de muestras de la señal de entrada
    XX = np.zeros(N, dtype=np.complex128)  # Inicializamos un arreglo para almacenar la DFT

    for k in range(N):
        XX[k] = 0
        for n in range(N):
            XX[k] += xx[n] * np.exp(-2j * np.pi * k * n / N)
    return XX/N

<p style="font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px;">
    Calculamos la DFT de la señal
</p>

In [56]:
XX = mi_funcion_DFT( xx )

# Calcular la magnitud del espectro en frecuencia
X_mag = np.abs(XX)

# Crear un vector de frecuencia
df  = fs/N
frequencies = np.arange(start=0, stop=(N)*df, step=df)
bfrec = frequencies <= fs/2
# Visualizar el espectro en frecuencia
psd.plot_signal(frequencies[bfrec], 10*np.log10(2*np.abs(XX[bfrec])**2),'PSD', color='#CD5C5C', xlabel='Frecuencia [f]', ylabel='Amplitud [dB]')

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


<div style="text-align: center;">
  <span style="border: 2px solid white; padding: 5px; display: inline-block; background-color: #CD5C5C; font-family: 'American Typewriter', 'Courier New', Courier, 'Lucida Sans Typewriter', 'Lucida Typewriter', monospace; font-size: 20px; color: white; font-weight: bold;">
    PSD
  </span>
</div>
<div style="max-width: 100%; text-align: center;">
  <img src="PSD.png" alt="Texto alternativo" style="max-width: 100%;">
</div>